In [3]:
import csv
import sys,os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
from matplotlib.colors import rgb2hex, Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.patches import Polygon
from matplotlib.colorbar import ColorbarBase
import seaborn as sns
from pandas import DataFrame
%matplotlib inline
import pylab
from pylab import *
import sklearn
from sklearn import tree, ensemble
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [78]:
orders = pd.read_csv('orders.txt', sep='\t',index_col=0)
zipcensus = pd.read_csv('zipcensus.txt', sep='\t', index_col=0)
zipcounty = pd.read_csv('zipcounty.txt', sep='\t', index_col=0, encoding='latin-1')
customer = pd.read_csv('customer.txt', sep='\t',index_col=0)
zipcensus.head(1)

,state,population,housingunits,landareameters,waterareameters,landareamiles,waterareamiles,latitude,longitude,numstates,...,hhuofuelnone,hhuoplumbingcomplete,hhuoplumbinglacking,hhumediancashrent,hhunmarried,hhunmarriedmm,hhunmarriedmf,hhunmarriedff,hhunmarriedfm,hhother
zipcode,,,,,,,,,,,,,,,,,,,,,
601,PR,19143,6715,172731389,1082233,66.691965,0.417853,18.180103,-66.749472,1,...,0.622561,0.948771,0.051229,214,0.024936,0.001866,0.010178,0.009839,0.003053,0.978456


In [16]:
#Tworzymy tabelę łączoną orders_rozsz i customer, połączone za pomocą klucza customerid.
OrdersCustomer = orders.join(customer, on='customerid')
OrdersCustomer.head(1)

,customerid,campaignid,orderdate,city,state,zipcode,paymenttype,totalprice,numorderlines,numunits,householdid,gender,firstname
orderid,,,,,,,,,,,,,
1002854,45978,2141,13.10.2009 00:00,NEWTON,MA,2459,VI,190.0,3,3,18991234.0,M,LOUIS


In [31]:
#OrdersCustomerZipcensus = OrdersCustomer.join(zipcensus, on='zipcode')
OrdersCustomerZipcensus = pd.concat([OrdersCustomer, zipcensus], sort=False)
OrdersCustomerZipcensus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225021 entries, 1002854 to 99950
Columns: 124 entries, customerid to hhother
dtypes: float64(114), object(10)
memory usage: 214.6+ MB


In [34]:
#OrdersCustomerZipcensus = OrdersCustomer.join(zipcensus, on='zipcode')
OrdersCustomerZipcensus = pd.concat([orders, customer, zipcensus], sort=False)
df1 = OrdersCustomerZipcensus['orderid', 'zipcode', 'paymenttype', 'totalprice', 'numunits', 'city', 'state', 'hhmedincome', 'purban', 'gender']
df1.head(1)

KeyError: ('orderid', 'zipcode', 'paymenttype', 'totalprice', 'numunits', 'city', 'state', 'hhmedincome', 'purban', 'gender')

In [59]:
NewData1 = DataFrame(OrdersCustomerZipcensus, columns=['zipcode', 'paymenttype', 'totalprice', 'numunits', 'city', 'state', 'hhmedincome', 'purban', 'gender'])
NewData

,zipcode,paymenttype,totalprice,numunits,city,state,hhmedincome,purban,gender
1002854,2459,VI,190.00,3.0,NEWTON,MA,NaN,NaN,NaN
1002855,10804,VI,10.00,1.0,NEW ROCHELLE,NY,NaN,NaN,NaN
1002856,33137,AE,35.22,2.0,MIAMI,FL,NaN,NaN,NaN
1002857,7073,AE,10.00,1.0,E RUTHERFORD,NJ,NaN,NaN,NaN
1002886,21218,VI,10.00,1.0,BALTIMORE,MD,NaN,NaN,NaN
1002887,6853,AE,10.00,1.0,ROWAYTON,CT,NaN,NaN,NaN
1002888,46240,VI,10.00,1.0,INDIANAPO11,IN,NaN,NaN,NaN
1002889,10570,MC,10.00,1.0,PLEASANTVILLE,NY,NaN,NaN,NaN
1002890,18301,DB,29.68,2.0,EAST STROUDSBURG,PA,NaN,NaN,NaN
1003004,60073,DB,19.68,1.0,ROUND LAKE PARK,IL,NaN,NaN,NaN


In [29]:
NewData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225021 entries, 1002854 to 99950
Data columns (total 10 columns):
orderid        0 non-null float64
zipcode        192837 non-null object
paymenttype    192983 non-null object
totalprice     192983 non-null float64
numunits       192983 non-null float64
city           192964 non-null object
state          223902 non-null object
hhmedincome    32038 non-null float64
purban         32038 non-null float64
gender         173355 non-null object
dtypes: float64(5), object(5)
memory usage: 18.9+ MB


In [40]:
NewData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414580 entries, 1002854 to 99950
Data columns (total 9 columns):
zipcode        192837 non-null object
paymenttype    192983 non-null object
totalprice     192983 non-null float64
numunits       192983 non-null float64
city           192964 non-null object
state          223902 non-null object
hhmedincome    32038 non-null float64
purban         32038 non-null float64
gender         173355 non-null object
dtypes: float64(4), object(5)
memory usage: 31.6+ MB


In [57]:
NewData.dropna(subset = ['gender'])

,zipcode,paymenttype,totalprice,numunits,city,state,hhmedincome,purban,gender
174596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M
68239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M
10203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F
174288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F
68099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M
174257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F
173506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M
51886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F
143249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F
174650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M


In [60]:
OrdersCustomer.dropna()

,customerid,campaignid,orderdate,city,state,zipcode,paymenttype,totalprice,numorderlines,numunits,householdid,gender,firstname
orderid,,,,,,,,,,,,,
1002854,45978,2141,13.10.2009 00:00,NEWTON,MA,2459,VI,190.00,3,3,18991234.0,M,LOUIS
1002855,125381,2173,13.10.2009 00:00,NEW ROCHELLE,NY,10804,VI,10.00,1,1,36173848.0,M,ANTHONY
1002857,130980,2173,14.10.2009 00:00,E RUTHERFORD,NJ,7073,AE,10.00,1,1,36173923.0,F,MARIA
1002887,106150,2173,15.10.2009 00:00,ROWAYTON,CT,6853,AE,10.00,1,1,36185316.0,M,CRAIG
1002888,27805,2173,15.10.2009 00:00,INDIANAPO11,IN,46240,VI,10.00,1,1,18368062.0,F,SUSAN
1002889,24546,2173,15.10.2009 00:00,PLEASANTVILLE,NY,10570,MC,10.00,1,1,19319923.0,M,JOE
1002890,43783,2173,15.10.2009 00:00,EAST STROUDSBURG,PA,18301,DB,29.68,2,2,19130968.0,M,ROBERT
1003004,15688,2173,15.10.2009 00:00,ROUND LAKE PARK,IL,60073,DB,19.68,1,1,20542726.0,M,BILL
1003044,130970,2141,22.11.2010 00:00,BLOOMFIELD,NJ,7003,AE,10.00,1,1,36191206.0,F,WINIFRED


In [62]:
cd1 = OrdersCustomer.join(zipcensus, on='zipcode')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [70]:
OrdersCustomer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192983 entries, 1002854 to 1643157
Data columns (total 13 columns):
customerid       192983 non-null int64
campaignid       192983 non-null int64
orderdate        192983 non-null object
city             192964 non-null object
state            191864 non-null object
zipcode          192837 non-null object
paymenttype      192983 non-null object
totalprice       192983 non-null float64
numorderlines    192983 non-null int64
numunits         192983 non-null int64
householdid      189559 non-null float64
gender           173355 non-null object
firstname        186644 non-null object
dtypes: float64(2), int64(4), object(7)
memory usage: 20.6+ MB


In [64]:
zipcensus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32038 entries, 601 to 99950
Columns: 112 entries, state to hhother
dtypes: float64(90), int64(18), object(4)
memory usage: 27.6+ MB


In [92]:
x = OrdersCustomer.set_index('zipcode')
cd1 = zipcensus.join(x, on='zipcode')

ValueError: columns overlap but no suffix specified: Index(['state'], dtype='object')

In [117]:
orders1 = pd.read_csv('orders1.txt', sep='\t',index_col=0)
OrdersCustomer = orders1.join(customer, on='customerid')
k = OrdersCustomer.join(zipcensus, on='zipcode', how='left', lsuffix='_left', rsuffix='_right')
n = k.dropna()
NewData1 = DataFrame(n, columns=['zipcode', 'paymenttype', 'totalprice', 'numunits', 'city', 'state_right', 'state_right', 'hhmedincome', 'purban', 'gender'])
NewData1

,zipcode,paymenttype,totalprice,numunits,city,state_right,state_right,hhmedincome,purban,gender
orderid,,,,,,,,,,
1003685,10004,VI,17.95,1,NEW YORK,NY,NY,101868.0,1.000000,F
1003576,10004,AE,22.05,1,NEW YORK,NY,NY,101868.0,1.000000,M
1004077,10004,DB,10.00,1,NEW YORK,NY,NY,101868.0,1.000000,M
1000147,10004,AE,10.00,1,NEW YORK,NY,NY,101868.0,1.000000,F
1002147,10004,AE,22.05,1,NEW YORK,NY,NY,101868.0,1.000000,M
1008498,10004,VI,14.95,1,NEW YORK,NY,NY,101868.0,1.000000,F
1005823,10004,MC,29.68,2,NEW YORK,NY,NY,101868.0,1.000000,M
1016816,10004,AE,175.00,1,NEW YORK,NY,NY,101868.0,1.000000,M
1015973,10004,AE,59.76,5,NEW YORK,NY,NY,101868.0,1.000000,M


In [111]:
y = OrdersCustomer.dropna()
z = zipcensus.dropna()
k = y.join(z, on='zipcode')

ValueError: columns overlap but no suffix specified: Index(['state'], dtype='object')

In [83]:
from decimal import Decimal
orders1['zipcode'] = orders1.zipcode.astype('int64')

ValueError: invalid literal for int() with base 10: 'M5R3R'

In [ ]:
zipcensus = pd.read_csv('zipcensus.txt', sep='\t', index_col=0)
zipcounty = pd.read_csv('zipcounty.txt', sep='\t', index_col=0, encoding='latin-1')
customer = pd.read_csv('customer.txt', sep='\t',index_col=0)
orders1 = pd.read_csv('orders1.txt', sep='\t',index_col=0)

OrdersCustomer = orders1.join(customer, on='customerid')
k = OrdersCustomer.join(zipcensus, on='zipcode', how='left', lsuffix='_left', rsuffix='_right')
n = k.dropna()
NewData1 = DataFrame(n, columns=['zipcode', 'paymenttype', 'totalprice', 'numunits', 'city', 'state_right', 'state_right', 'hhmedincome', 'purban', 'gender'])
NewData1